In [2]:
import json
data = sc.textFile('hdfs://master:54310/exp_2/ml_data_without_urls'). \
    map(lambda x: json.loads(x))

In [17]:
data.flatMap(lambda doc: doc['Labels']). \
        distinct().count()

99546

In [16]:
all_labels = data.flatMap(lambda doc: doc['Labels']). \
        distinct().filter(lambda doc: 'блог компан' in doc)

In [12]:
words_dict = all_labels.map(lambda l: (l, l.replace('блог компан', '').strip())). \
        filter(lambda p: p[1] != '').collectAsMap()

In [30]:
def replace_labels(doc):
    doc['Labels'] = list(set(words_dict.get(l,l) for l in doc['Labels']))
    return doc

In [31]:
data.flatMap(lambda doc: doc['Labels']).map(lambda l: words_dict.get(l,l)). \
        distinct().count()

98926

In [32]:
data_n = data.map(replace_labels)

In [33]:
data_n.count()

114248

In [35]:
data_n.map(json.dumps).repartition(4). \
    saveAsTextFile('hdfs://master:54310/exp_3/ml_data_without_urls_and_blogs')